In [1]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.executor.memory", "6g") \
    .config("spark.driver.memory", "10g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size","6g") \
    .config("spark.sql.files.maxPartitionBytes", "128m") \
    .config("spark.sql.shuffle.partitions", "300") \
    .getOrCreate()


pd.set_option('display.max_columns', 500)

!wget "https://obis-datasets.ams3.digitaloceanspaces.com/exports/obis_20231025.parquet"

24/03/02 13:23:32 WARN Utils: Your hostname, Alexanders-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.16.140.184 instead (on interface en0)
24/03/02 13:23:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 13:23:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read the parquet file
df = spark.read.parquet('/Users/helloworld/Downloads/obis_20231025.parquet')
obis_columns = ["marine", "class", "genus", "species", "variety", "individualCount", "eventDate", "eventTime", "year", "month", "day", "country", "locality", "decimalLongitude", "decimalLatitude"]

# # Select the relevant columns and repartition
obis_df = df.select(obis_columns)
obis_df.registerTempTable('obis_data')
obis_df.repartition(300).write.mode('overwrite').parquet('obis.parquet')